<a href="https://colab.research.google.com/github/rohanyadav9/SMARTRECRUITER/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-generativeai langchain faiss-cpu sentence-transformers pypdf2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.4 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import drive

# Loading and Indexing Data

In [ ]:
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    print("Please restart the runtime and try again.")

# ==== LOAD PDFs ====
def load_pdfs(pdf_folder_path):
    docs = []
    if not os.path.exists(pdf_folder_path):
        print(f"Folder not found: {pdf_folder_path}")
        return docs
    for fname in os.listdir(pdf_folder_path):
        if fname.lower().endswith(".pdf"):
            path = os.path.join(pdf_folder_path, fname)
            try:
                reader = PdfReader(path)
                text = "\n".join(page.extract_text() or "" for page in reader.pages)
                docs.append({"filename": fname, "text": text})
            except Exception as e:
                print(f"Error reading {fname}: {e}")
    return docs

pdf_folder = "/content/drive/MyDrive/Python_document"
pdf_docs = load_pdfs(pdf_folder)
print(f"Loaded {len(pdf_docs)} PDFs.")

Error mounting Google Drive: Error: credential propagation was unsuccessful
Please restart the runtime and try again.
Loaded 5 PDFs.


# **Splitting Data into Chunks**

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
)

docs_with_chunks = []
for doc in pdf_docs:
    chunks = splitter.split_text(doc["text"])
    for i, chunk in enumerate(chunks):
        docs_with_chunks.append({
            "text": chunk,
            "meta": {"source": doc["filename"], "chunk": i}
        })

In [ ]:
len(docs_with_chunks)## No. of chunks created

5473

In [ ]:
docs_with_chunks[876] ## Randomly checking data in any of the chunks.

{'text': 'Itiswrittenas None.\n4.14.8TheEllipsisObject\nThisobjectiscommonlyusedbyslicing(seeslicings). Itsupportsnospecialoperations. Thereisexactlyoneellipsis\nobject,named Ellipsis (abuilt-inname). type(Ellipsis)() producesthe Ellipsis singleton.\nItiswrittenas Ellipsis or....\n4.14.9TheNotImplementedObject\nThis object is returned from comparisons and binary operations when they are asked to operate on types\nthey don’t support. See comparisons for more information. There is exactly one NotImplemented object.\ntype(NotImplemented)() producesthesingletoninstance.\nItiswrittenas NotImplemented .\n102 Chapter4. Built-inTypes\nThePythonLibraryReference,Release3.13.3\n4.14.10InternalObjects\nSeetypesforthisinformation. Itdescribesstackframeobjects,tracebackobjects,andsliceobjects.\n4.15SpecialAttributes\nTheimplementationaddsafewspecialread-onlyattributestoseveralobjecttypes,wheretheyarerelevant. Someof\nthesearenotreportedbythe dir()built-infunction.\ndefinition. __name__\nThenameofthe

# Creation of Vector DB

In [ ]:
from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

class MySentenceTransformerEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return model.encode(texts).tolist()

    def embed_query(self, text):
        return model.encode([text])[0].tolist()

embedding = MySentenceTransformerEmbeddings()

documents = [
    Document(page_content=d["text"], metadata=d["meta"])
    for d in docs_with_chunks
]

vector_db = FAISS.from_documents(documents, embedding=embedding)

print("Vector store created!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Vector store created!


In [ ]:
vector_db.index_to_docstore_id ## Every Chunks got there individual Id's.

{0: '3cecea5a-f0c7-46be-9f7d-d6b59247f62f',
 1: 'edf746b5-3e95-4592-a5ca-28a631273dff',
 2: 'afdf7d8d-ba9a-47a4-8243-5fe5aef6f822',
 3: 'f7409e77-9630-411e-bf86-4e02825206bd',
 4: 'ad5aa9f1-8d60-4dff-ad17-9a4efac22df2',
 5: '154e29eb-a1a7-4281-9b4c-af76ab58448f',
 6: 'e75184ca-9565-4f25-b613-d83d06aa15fb',
 7: 'b9f12de6-cd56-482e-8c0f-f1a03eb77cb1',
 8: '4cd24b5b-2167-4ac2-af1c-1b08bb68fec5',
 9: 'c40e79a1-9060-4bb2-9274-4f58d3dfedf9',
 10: '53dd629b-95ef-4afb-bd86-c6c786e292b4',
 11: '260e5ddb-369a-40b8-b875-5a2cbff89617',
 12: '72b7fdc6-35dc-40fe-b44d-c51bbcf587e9',
 13: '65c9ddeb-dc26-49c8-88aa-b39505817bda',
 14: '61f5dfb1-502d-4237-b8f5-61de319a13d0',
 15: 'be4733e0-a9d9-49cb-9322-e1dcb9a9048a',
 16: 'c82d5944-d5ba-4758-b4ee-bac3491cd4c3',
 17: 'd729ef80-4c95-4fb1-80b7-153525051860',
 18: '89ee95f4-5fec-4805-8594-d28896ac9acd',
 19: '26dada63-b9e4-4a95-881a-7c6b92d61875',
 20: '7c938740-a86e-41d5-9e07-f186f812feda',
 21: '8c532eb7-d2eb-458f-8c6f-c0df252e55ed',
 22: 'f3e2f9cf-e374-

In [ ]:
vector_db.get_by_ids(['99ffd5e1-4727-495a-9df7-0c1752a9a401'])## Randomly checking whats stores in one the ID.

[Document(id='99ffd5e1-4727-495a-9df7-0c1752a9a401', metadata={'source': 'reference.pdf', 'chunk': 195}, page_content='The Python Language Reference, Release 3.13.3\n6.7Binary arithmetic operations\nThe binary arithmetic operations have the conventional priority levels. Note that some of these operations also\napplytocertainnon-numerictypes. Apartfromthepoweroperator,thereareonlytwolevels,oneformultiplicative\noperatorsandoneforadditiveoperators:\nm_expr ::= u_expr |m_expr "*" u_expr |m_expr "@" m_expr |\nm_expr "//" u_expr |m_expr "/" u_expr |\nm_expr "%" u_expr\na_expr ::= m_expr |a_expr "+" m_expr |a_expr "-" m_expr\nThe*(multiplication)operatoryieldstheproductofitsarguments. Theargumentsmusteitherbothbenumbers,or\noneargumentmustbeanintegerandtheothermustbeasequence. Intheformercase,thenumbersareconvertedtoa\ncommontypeandthenmultipliedtogether. Inthelattercase,sequencerepetitionisperformed;anegativerepetition\nfactoryieldsanemptysequence.\nThisoperationcanbecustomizedusingthespeci

# ReTriver

In [ ]:
retriver = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
retriver.invoke('What is string?')

[Document(id='d23e4772-4438-4218-a16e-b7faae3664cd', metadata={'source': 'faq.pdf', 'chunk': 85}, page_content='3.8Whyisjoin()astringmethodinsteadofalistortuplemethod?\nStrings became much more like other standard types starting in Python 1.6, when methods were added which give\nthe same functionality that has always been available using the functions of the string module. Most of these new\nmethodshavebeenwidelyaccepted,buttheonewhichappearstomakesomeprogrammersfeeluncomfortableis:\n",".join([ \'1\',\'2\',\'4\',\'8\',\'16\'])\nwhichgivestheresult:\n"1, 2, 4, 8, 16 "\nTherearetwocommonargumentsagainstthisusage.\nThefirstrunsalongthelinesof: “Itlooksreallyuglyusingamethodofastringliteral(stringconstant)”,towhichthe\nansweristhatitmight,butastringliteralisjustafixedvalue. Ifthemethodsaretobeallowedonnamesboundto\nstringsthereisnologicalreasontomakethemunavailableonliterals.\nThe second objection is typically cast as: “I am really telling a sequence to join its members together with a str

In [ ]:
query = "what is string ?"
retrived_docs = retriver.invoke(query)

In [ ]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

context_text = format_docs(retrived_docs)

# Building of prompt

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
prompt = PromptTemplate(
    template="""You are a helpful assistant.
     Answer Only from the provided context .
     If context is insufficient, just say you don't know
     Context : {context_text}
     Question : {query}

     """ ,
     input_variables=["context_text" , "query"]
)


# Building of model

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key = "Google_api_key")

In [ ]:
final_prompt = prompt.format_prompt(context_text=context_text, query=query)
response = llm.invoke(final_prompt.to_messages())
print(response.content)

Based on the provided text, a string is a type of Python object.  A string literal is a fixed value.  Strings can be single-quoted or double-quoted. Triple-quoted strings allow unescaped single and double quotes within the string and can span multiple lines.


# Building of Chain

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parallel_chain = RunnableParallel({
    'context_text' : retriver | RunnableLambda(format_docs),
    'query' : RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('what is tuples?')

{'context_text': 'onlyworksinsideaclassdefinition,asthecompilerfillsinthenecessarydetailstocorrectlyretrievetheclass\nbeingdefined,aswellasaccessingthecurrentinstanceforordinarymethods.\nForpracticalsuggestionsonhowtodesigncooperativeclassesusing super(),seeguidetousingsuper() .\nclass tuple\nclass tuple (iterable )\nRather than being a function, tupleis actually an immutable sequence type, as documented in Tuplesand\nSequence Types — list, tuple, range .\nclass type (object )\n30 Chapter2. Built-inFunctions\nThePythonLibraryReference,Release3.13.3\nclass type (name,bases,dict,**kwds )\nWithoneargument,returnthetypeofan object. Thereturnvalueisatypeobjectandgenerallythesameobject\nasreturnedby object.__class__ .\nTheisinstance() built-infunctionisrecommendedfortestingthetypeofanobject,becauseittakessub-\nclassesintoaccount.\nWiththreearguments, returnanewtypeobject. Thisisessentiallyadynamicformofthe classstatement.\nThenamestring is the class name and becomes the __name__ attribute. T

In [ ]:
parse = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parse

In [ ]:
main_chain.invoke('what is tuples?')

"`tuple` is an immutable sequence type.  Unlike most Python containers, it's common for tuples to contain elements of different types.  Tuple accepts any number of type arguments.  Empty tuples are constructed with empty parentheses `()`; a tuple with one item needs a trailing comma after the value."

In [ ]:
main_chain.invoke('what is dictonary?')

'An associative array, where arbitrary keys are mapped to values.  The keys can be any object with `__hash__()` and `__eq__()` methods. Called a hash in Perl.'